## Primeros pasos con TensorFlow

#### Goals
* Aprender las bases de TensorFlow 
* Usar la clase LinearRegressor para predecir el precio mediano
* Evaluar el rendimiento del modelo haciendo uso del RMSE 
* Mejorar la exactitud del modelo ajustando los hiperparametros

In [1]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [2]:
#cargamos el dataset
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

In [3]:
california_housing_dataframe.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.3,34.2,15.0,5612.0,1283.0,1015.0,472.0,1.5,66900.0
1,-114.5,34.4,19.0,7650.0,1901.0,1129.0,463.0,1.8,80100.0
2,-114.6,33.7,17.0,720.0,174.0,333.0,117.0,1.7,85700.0
3,-114.6,33.6,14.0,1501.0,337.0,515.0,226.0,3.2,73400.0
4,-114.6,33.6,20.0,1454.0,326.0,624.0,262.0,1.9,65500.0


In [4]:
#vamos a mover los datos de manera randomica
california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))

#vamos a tratar los valores del median_house_value en miles para que todo sea mas facil
california_housing_dataframe["median_house_value"] /= 1000.0

#imprimimos el dataset
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
4725,-118.1,34.0,37.0,1743.0,348.0,1328.0,354.0,3.1,162.8
7365,-118.3,34.1,26.0,3977.0,1050.0,1720.0,935.0,3.4,364.5
14594,-122.2,37.5,33.0,3687.0,852.0,3091.0,852.0,2.7,162.6
3039,-117.8,33.7,15.0,2099.0,322.0,873.0,307.0,8.0,328.0
1409,-117.2,32.8,31.0,3064.0,575.0,1476.0,549.0,3.7,175.9
...,...,...,...,...,...,...,...,...,...
13734,-122.0,37.4,17.0,2070.0,278.0,797.0,275.0,8.6,411.2
7819,-118.4,34.0,38.0,2521.0,647.0,1091.0,597.0,4.1,322.9
3309,-117.9,33.9,15.0,1898.0,476.0,1766.0,455.0,2.5,158.5
6729,-118.3,34.0,43.0,2660.0,672.0,2133.0,588.0,1.8,107.3


In [5]:
#veremos un breve resumen estadistico del dataset
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


### Creacion del primer modelo 
Trataremos de predecir el valor de median_house_value (será nuestro target), usaremos el feature de total_rooms como atributo de entrada.
* Para realizar esta tarea usaremos la interfaz de LinearRegressor que proporciona la API de Estimator de TensorFlow. Esta API ocupa en gran medida el ajuste del modelo de bajo nivel.


<br>

#### Paso 1: Definimos atributos y configuramos las columnas de atributos
Para importar nuestros datos de entrenamiento en TensorFlow, debemos especificar que tipo de datos contiene cada atributo. Hay dos tipos de datos que usaremos:
* <b>Datos categoricos</b>: Datos que son textuales, en este dataset no hay este tipo de datos. (Ej: estilo de la casa)
* <b>Datos numéricos</b>: Datos que son un numero (entero o flotante) y que quieres tratar como un numero. A veces se quiere tratar los datos numericos (codigo postal) como si fuera categorico.

<br>
En TF se indica el tipo de datos a traves de un constructo denominado columna de atributos. Estos solo almacenan una descripcion de los datos de los atributos; no contienen los datos de los atributos en si.

Para comenzar usaremos solo un atributo de entrada numerica, total_rooms. A continuacion extraeremos los datos de total_rooms del dataset y definiremos la columna de atributos con numeric_column, el cual especifica que los datos son numericos.


In [10]:
#Definimos el atributo entrada: total_rooms
my_feature = california_housing_dataframe[["total_rooms"]]

In [11]:
my_feature

,total_rooms
4725,1743.0
7365,3977.0
14594,3687.0
3039,2099.0
1409,3064.0
...,...
13734,2070.0
7819,2521.0
3309,1898.0
6729,2660.0


#### Paso 2:  Definimos el objetivo

In [12]:
#Configuramos nuestra columna de caractericas numerico para total_rooms
feature_columns = [tf.feature_column.numeric_column("total_rooms")]

In [15]:

feature_columns

[_NumericColumn(key='total_rooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [14]:
#Definimoa la etiqueta
targets = california_housing_dataframe["median_house_value"]

In [16]:
targets

4725    162.8
7365    364.5
14594   162.6
3039    328.0
1409    175.9
         ... 
13734   411.2
7819    322.9
3309    158.5
6729    107.3
3401    345.4
Name: median_house_value, Length: 17000, dtype: float64

In [17]:
targets.size

17000

#### Paso 3:  Configuramos el regresor lineal
Entrenaremos el modelo mediante el GradientDescentOptimizer, el cual usa SGD de minilote. El argumento learning_rate controla el tamaño del paso de gradiente.

<b>Nota:</b> Para estar seguros tambien haremos recorte de gradientes a nuestro optimizador a traves de <b>clip_gradients_by_norm</b>. Este recorte nos asegura que la magnitud de los gradientes no sea demasiado grande en el entrenamiento, lo cual puede provocar que falle el descenso de gradientes.


In [21]:
#Usamos la gradiente descendiente como optimizador para entrenar el modelo
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer,5.0)

In [22]:
#Configuramos el modelo de regresion lineal con nuestra columna de caracteristicas y optimizador
linear_regressor = tf.estimator.LinearRegressor(feature_columns =  feature_columns, optimizer = my_optimizer)

#### Paso 4: Definimos la funcion de entrada
Para importar nuestros datos de viviendas a nuestro Modelo, tenemos que definir una funcion de entrada, que instruye a TensorFlow como realizar el procesamiento pevio de los datos, ademas de como:
* Organizarlos por lotes
* Aleatorizarlos
* Repetirlos
Durante el entrenamiento del modelo.

<br>
<ol>
<li> Primero convertiremos nuestros datos de atributos de pandas a un diccionario de matrices de Numpy. </li>

<li> Usamos la API de conjunto de datos de TF para construir un objeto de conjunto de datos a partir de nuestros datos y luegom dividir nuestros datos en lotes de batch_size para que se repitan para el numero especificado de repeticiones (num_epochs). </li>
<li> Nuestra funcion de entrada construye una variable de iteracion para el conjunto de datos y devuelve el siguiente lote de datos al regresor lineal.</li>

In [23]:
def input_fn(features,targets,batch_size=1,shuffle=True,num_epochs=None):
    '''
    Entrenamiento de un modelo de regresion lineal de una caracteristica.
    Argumentos:
        features: Dataframe (pandas) de las caracteristicas
        targets: Dataframe (pandas) de los objetivos
        batch_size: Tamaño de los lotes que seran pasados al modelo
        shuffle: Verdadero o Falso, para barajear los datos.
        num_epochs: Numero de epochs para los cuales se deben repetir los datos. None = repetir indefinitivamente.
    
    Devuelve:
        Tuplas de (features,labels) para el siguiente lote de datos,
    '''
    
    #Convertimos los datos pandas en diccionarios de arrays numpy.
    features = {key:np.array(value) for key,value in dict(features).item()}
    
    #Constuimos un dataset, y configuramos el batching/repeating
    ds = Dataset.from_tensor_slices((features,targets)) #ojo: limite de 2GB?
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    #Barajeamos los datos, si es el caso
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
    
    #Devolvemos el siguiente lote de datos
    features,labels = ds.make_one_shot_iterator().get_next()
    return features,labels
    